# Guided Project: Building a Spam Filter with Naive Bayes

To classify messages as spam or non-spam, we saw in the previous mission that the computer:

- Learns how humans classify messages.
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
- Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from this [link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on this [page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

In [1]:
# Import libraries
import pandas as pd

In [2]:
# Load dataset
sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
print(sms.shape)
# Print what percentage is spam to ham
sms['Label'].value_counts(normalize=True) * 100

(5572, 2)


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

We have around five thousand records which most of the messages are ham(means non-spam)

However, before creating it, it's very helpful to first think of a way of testing how well it works. When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- A training set, which we'll use to "train" the computer how to classify messages.
- A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

In [3]:
# Randomize the dataset
data_randomized = sms.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Training/Test split
training = data_randomized[:training_test_index].reset_index(drop=True)
test = data_randomized[training_test_index:].reset_index(drop=True)

# Print the percentage of spam and ham in both training and test set
print("Training Set Percentage with {} # of records".format(len(training)))
print(training['Label'].value_counts(normalize=True) * 100)
print("\n")
print("Test Set Percentage with {} # of records".format(len(test)))
test['Label'].value_counts(normalize=True) * 100

Training Set Percentage with 4458 # of records
ham     86.54105
spam    13.45895
Name: Label, dtype: float64


Test Set Percentage with 1114 # of records


ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The results give the similar percentages both training and test set to the full dataset. The next step is to use the training set to teach the algorithm to classify new messages.

We need to convert the SMS column into this and do some data cleaning.

![image](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_2.png)

In [4]:
# By doing data cleaning process, remove punctuation and bring all words to lower case
training['SMS'] = training['SMS'].str.replace('\W', ' ').str.lower()
# Print the first five records to see if it's clean
training['SMS'].head()

0                         yep  by the pretty sculpture
1        yes  princess  are you going to make me moan 
2                           welp apparently he retired
3                                              havent 
4    i forgot 2 ask ü all smth   there s a card on ...
Name: SMS, dtype: object

Now, it's clean. Let's make a vocabulary that should be in a python list containing all the unique words across all messsages

In [5]:
# list comprehension
vocabulary = [item for row in training['SMS'] for item in row.split()]
# traditional
# for row in training['SMS']:
#     words = row.split()
#     for item in words:
#         vocabulary.append(item)

# Transform list into set that removes the duplications
print("{} of records vocabulary with duplications".format(len(vocabulary)))
vocabulary = set(vocabulary)
# Transform back to list from set
print("{} of records vocabulary w/o duplications".format(len(vocabulary)))
vocabulary = list(vocabulary)

72427 of records vocabulary with duplications
7783 of records vocabulary w/o duplications


Create a dictionary of word counts per sms and convert to dataframe with concatenate the training dataset.

We start by initializing a dictionary named word_counts_per_sms, where each key is a unique word (a string) from the vocabulary, and each value is a list of the length of training set, where each element in the list is a 0.

- The code [0] * 5 outputs [0, 0, 0, 0, 0]. So the code [0] * len(training_set['SMS']) outputs a list of the length of training_set['SMS'], where each element in the list will be a 0.

We loop over training_set['SMS'] using at the same time the enumerate() function to get both the index and the SMS message (index and sms).

Using a nested loop, we loop over sms (where sms is a list of strings, where each string represents a word in a message).
We incremenent word_counts_per_sms[word][index] by 1.

In [6]:
# Splitting the words
training['SMS'] = training['SMS'].str.split()

In [7]:
training['SMS'].head()

0                    [yep, by, the, pretty, sculpture]
1    [yes, princess, are, you, going, to, make, me,...
2                      [welp, apparently, he, retired]
3                                             [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS, dtype: object

In [8]:
# Provided by the project
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
    
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [9]:
# Concatenate the dataframes
training_set_clean = pd.concat([training, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


Recall that the Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

P(Spam|w1,w2,...,wn)∝P(Spam)⋅∏i=1nP(wi|Spam)
P(Ham|w1,w2,...,wn)∝P(Ham)⋅∏i=1nP(wi|Ham)

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, recall that we need to use these equations:

P(wi|Spam)=Nwi|Spam+α / NSpam+α⋅NVocabulary

P(wi|Ham)=Nwi|Ham+α / NHam+α⋅NVocabulary

In [10]:
# Find the P(Spam) and P(Ham)
training_set_clean['Label'].value_counts()

ham     3858
spam     600
Name: Label, dtype: int64

In [11]:
p_spam = training_set_clean['Label'].value_counts(normalize=True)['spam']
p_ham = training_set_clean['Label'].value_counts(normalize=True)['ham']

In [12]:
# Assign the number of vocabulary
n_vocabulary = len(vocabulary)
print("{} is Nvocabulary".format(n_vocabulary))

7783 is Nvocabulary


In [13]:
# Make a seprate series for ham and spam
ham_only = training.loc[training['Label'] == 'ham', 'SMS']
spam_only = training.loc[training['Label'] == 'spam', 'SMS']

In [14]:
# Make a words of list from the separation
vocabulary_ham = [item for row in ham_only for item in row]
n_ham = len(vocabulary_ham)

vocabulary_spam = [item for row in spam_only for item in row]
n_spam = len(vocabulary_spam)

print("{} is Nspam".format(n_spam))
print("{} is Nham".format(n_ham))

15190 is Nspam
57237 is Nham


In [15]:
# Setting alpha of Laplace smoothing
alpha = 1

To solve P(wi|Spam) and P(wi|Ham), make a dictionaries where each key-value pair is a unique word

In [16]:
# Taken from the solution

# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Isolate spam and ham messages before starting the loop below
# Don't do this inside the loop, it'll add to code running time significantly
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and: 

If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.

If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.

If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [17]:
# Given by the project
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [18]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [19]:
classify('Sounds good, Tom, then see u there')

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


In [20]:
# Given by project
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [21]:
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

Accuracy=number of correctly classified messages / total number of classified messages

In [22]:
correct = 0
total = test.shape[0]
for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is almost perfect.

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.

If you want to keep working on this project, here's a few next steps you can take:

Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
Make the filtering process more complex by making the algorithm sensitive to letter case.
Get the project portfolio-ready by using a few tips from our style guide for data science projects.